# Lab Overview

### Create Application using OpenShift CLI
* In this lab part you will be creating an application using the oc command to push config files containing the application components such as deployments, services and routes
* This is usually done from one or more YAML files that define the application components

### YAML and OpenShift
* The definition of all the K8 objects are specified in a particular format. This format defines the object, its configurable parameters and how their definitions are to be presented
* There are many objects in OpenShift with many not often used
* The file content is called the manifest 

### YAML
* YAML (YAML Ain't Markup Language or Yet Another Markup Language) files are human-readable data serialization standard often used for configuration files and data exchange between languages
* They use a clear, concise syntax with indentation to represent data structures like lists and dictionaries, making them easy to read and write
* YAML files are used by K8 to create the objects
https://www.redhat.com/en/topics/automation/what-is-yaml


### K8 objects
* The structure of the yaml files is defined as a multi-layer dictionary
* This can be somewhat complex

https://kubernetes.io/docs/concepts/overview/working-with-objects/

https://yaml.org/


# View YAML files
* In this lab there are a number of pre-created YAML files that will be used to build the application 
* You will have a "quick" look at the files to see the basic format and layout
* A full understanding of all components is beyond the scope of this class 

## Open folder in VSCode
* In the VSCode file explorer, open the oc_manifests directory and explore the files
  * You will see files such as deployment.yml, service.yml, and route.yml
* Have a quick look through the yaml file comments but don't get too bogged down with syntax


## File View
The oc_manifests folder view from VSCode with the YAML files will look similar to the following diagram


![YAML list: ](images/yaml_files.png)

## Exploring Manifest YAML files
* As already stated we are not going to review content to any great depth
* As will be obvious from the names the files contain details of the deployment, service and routes

* Let's explore the deployment file
* Click on the deployment.yml file to open it

### Deployment ###
- It manages a set of identical pods, ensuring that the specified number of replicas are running at all times

---

```yaml
kind: Deployment
apiVersion: apps/v1
metadata:
  name: housley-bouncer-deployment                # The name of the Deployment
spec:
  replicas: 1                                     # The number of replicas to run
  selector:                                       # The selector for the pods that this Deployment manages
    matchLabels:
      app: housley-bouncer
  template:                                       # The template for the pods that this Deployment manages
    metadata:
      labels:                                     # The labels for the pods
        app: housley-bouncer
    spec:
      containers:
        - name: housley-bouncer-container         # The name of the container
          image: hlabs/housley-bouncer:latest     # The image to run in the container
          ports:
            - name: http                          # The port that the container is listening on
              containerPort: 8080
              protocol: TCP
```

---


### Key YAML file components

> kind: Deployment
>> Specifies the type of object

> metadata.name: 
>> what the deployment will be created called
>>> note that the hierarchy of the yaml is reflected in the name </br>
>>> so the highest level is metadata </br>
>>> the next level is name

> spec.template.spec.containers.image
>> this will be the location from where the image is pulled</br>
>>> when the image is downloaded for the first time it will be cached locally on the cluster</br>
>>> this caching is what allows the pods to be created so rapidly</br>
>>> the :latest tag defines the version </br>
>>> if the image in cache is out of date the new images (or part thereof) will be downloaded</br>
>>> notice the first part of the name includes the repository from which it is downloaded

> spec.template.spec.containers.ports
>> this is the application ports that are opened by the pod
>> note this is a YAML list denoted by the - at the beginning of the name 
>> this will match up to the pod's application configuration
>> this specific configuration details
>>> Protocol: tcp
>>> Port: 8080
>> As it stands this pod/port is not accessible from either inside the cluster or externally 
>>> this is where the service and route objects are used

* We will not explore as part of this lab the other files but feel free to open and consider the structure and syntax

* If you make any changes to the file it may result in the deployment failing 

* If you make changes and cannot recover the files can be copied 

## Deploy Application using YAML manifest files
* You have previously created an application from the OpenShift GUI
* In this step you will deploy the objects using the YAML files and the oc command
* The oc command has an "apply -f" option that allows a given manifest file to be pushed to OpenShift
* When running the oc command, an API call, with the required directives and parameters, is sent to the management cluster controller
* This connection is created during the oc login command

In [ ]:
### If you are logged out or in the wrong namespace - run the following ###
# Replace ## with your student number - Note: Values between 01 and 16 (note leading 0)
student_number = "##"


### Login to OpenShift ###
!oc login -u s$student_number -p'!@34QWer' https://api.ocp.ucsx.hl.dns:6443 --insecure-skip-tls-verify


### Change to your Project ###
!oc project ai-s$student_number

#### Expected Output 
The output should look similar to 

---

```
WARNING: Using insecure TLS client config. Setting this option is not supported!

Login successful.

You have access to 109 projects, the list has been suppressed. You can list all projects with 'oc projects'

Using project "ai-s02".
Already on project "ai-s02" on server "https://api.ocp.ucsx.hl.dns:6443".

```
---

In [ ]:
### Create deployment ###
!oc apply -f oc_manifests/deployment.yml

#### Expected Output 
The output should look similar to 

---

```
deployment.apps/housley-bouncer created

```
---


In [ ]:
### Create service ###
!oc apply -f oc_manifests/service.yml

The output should look similar to 

---

```
service/housley-bouncer-service created

```
---

In [ ]:
### Create route ###
!oc apply -f oc_manifests/route.yml

The output should look similar to 

---

```
route.route.openshift.io/housley-bouncer created

```
---

## Explore Objects in your namespace
### List all pods in your namespace
Observe that new objects have been created, including pods, services and deployments

In [ ]:
# get details of created objects
!oc get pods

The output should look similar to 

---

```
NAME                                           READY   STATUS      RESTARTS   AGE
ai-open-webui-deployment-02-5c89b5fbf4-bn4md   1/1     Running     0          95m
housley-bouncer-66d49b97db-rtqv5               1/1     Running     0          2m29s
housley-counter-1-build                        0/1     Completed   0          90m
housley-counter-cdf7f5bb5-q9b8b                1/1     Running     0          90m
```
---

In [ ]:
!oc get services

The output should look similar to 

---

```
NAME                      TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)    AGE
ai-open-webui-service-02  ClusterIP   172.30.165.98    <none>        8080/TCP   95m
housley-bouncer-service   ClusterIP   172.30.222.152   <none>        8080/TCP   2m12s
housley-counter           ClusterIP   172.30.156.22    <none>        8080/TCP   90m
```
---

In [ ]:
!oc get routes

The output should look similar to 

---

```
NAME                 HOST/PORT                                     PATH   SERVICES                  PORT        TERMINATION     WILDCARD
housley-bouncer      housley-bouncer-ai-s02.apps.ocp.ucsx.hl.dns          housley-bouncer-service    http-8080                   None
housley-counter      housley-counter-ai-s02.apps.ocp.ucsx.hl.dns          housley-counter            8080-tcp    edge/Redirect   None
openwebui-route-02   openwebui-route-02-ai-s02.apps.ocp.ucsx.hl.dns       ai-open-webui-service-02   8080        edge            None
```
---

### Describe a Pod
This command provides detailed information about a specific pod, including its status, containers, and events.
- You will need to copy the ID of a pod from the previous command.

In [ ]:
# Look at the details of your pod and how it relates to other objects
# ensure the xxxx are changed

!oc describe pod/housley-bouncer-xxxx

The output should look similar to 

---

```
NName:             housley-bouncer-66d49b97db-rtqv5
Namespace:        ai-s02
Priority:         0
Service Account:  default
Node:             ocp4.ucsx.hl.dns/172.16.26.24
Start Time:       Tue, 05 Aug 2025 16:47:11 +1000
Labels:           app=housley-bouncer
                  pod-template-hash=66d49b97db
Annotations:      k8s.ovn.org/pod-networks:
                    {"default":{"ip_addresses":["10.129.3.123/23"],"mac_address":"0a:58:0a:81:03:7b","gateway_ips":["10.129.2.1"],"routes":[{"dest":"10.128.0....
                  k8s.v1.cni.cncf.io/network-status:
                    [{
                        "name": "ovn-kubernetes",
                        "interface": "eth0",
                        "ips": [
                            "10.129.3.123"
                        ],
                        "mac": "0a:58:0a:81:03:7b",
                        "default": true,
                        "dns": {}
                    }]
                  openshift.io/scc: restricted-v2
                  seccomp.security.alpha.kubernetes.io/pod: runtime/default
Status:           Running
SeccompProfile:   RuntimeDefault
...
  Normal  Pulling         6m27s  kubelet            Pulling image "hlabs/housley_bouncer:latest"
  Normal  Pulled          6m23s  kubelet            Successfully pulled image "hlabs/housley_bouncer:latest" in 3.993s (3.993s including waiting). Image size: 217882182 bytes.
  Normal  Created         6m23s  kubelet            Created container: housley-bouncer-container
  Normal  Started         6m23s  kubelet            Started container housley-bouncer-container
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
```
---

### Show Logs of a Pod
This shows the logs of the main process running in the pod.

In [ ]:
!oc logs housley-bouncer-xxxx

The output should look similar to 

---

```

> housley-bouncer@1.0.0 start
> node src/server.js

Housley Bouncer app listening at http://0.0.0.0:8080

```
---

## Check the Application route is working
- Return to the OpenShift GUI and under your developer project > Topology view the newly created objects
- Then click on the route to see the application in your browser

